In [1]:
import torch
import cv2
torch.cuda.set_device(2)
_exp_name = "pre_semi_spin"
number_class=101
a_style='all spins'
a_type_1='all spins'
a_type_2='a+0.94_shi'

In [2]:
import numpy as np
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [3]:
myseed = 1023  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [4]:
import torchvision.models as models
# 載入預先訓練的 ResNet-18 模型
device = "cuda" if torch.cuda.is_available() else "cpu"  # "cuda" only when GPUs are available.
model = models.resnet18(pretrained=True)

# 將全連接層（分類層）修改為輸出 10 類的層
model.fc = nn.Linear(model.fc.in_features, number_class)

model = model.to(device)

/home/DomCheng/miniconda3/envs/mas/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/DomCheng/miniconda3/envs/mas/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
model_best = model.to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_{myseed}_best.ckpt"))
model_best.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:

import os
import re

#INTER_LINEAR – a bilinear interpolation (used by default)

def test_img(filename):
    Rh_data = []
    IMG_SIZE = 224
    labels = []
    
    
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # 組合完整的檔案路徑
        file_path = os.path.join(filename)

    # 讀取照片
    img = cv2.imread(file_path)

    # 確保成功讀取照片
    if img is not None:
        # 將照片 resize 到指定的尺寸
        resized_img = cv2.resize(img, (224, 224),interpolation=cv2.INTER_LINEAR)
        #resized_img=resized_img/255
        #resized_img[0] = (resized_img[0]-0.485)/0.229
        #resized_img[1] = (resized_img[1]-0.456)/0.224
        #resized_img[2] = (resized_img[2]-0.406)/0.225

        # 將 resize 後的照片加入列表
        Rh_data.append(resized_img)
    else:
        print(f"無法讀取照片: {file_path}")
                
    # 使用正則表達式匹配檔案名稱中的數字部分，並轉換為浮點數
    match = re.search(r'a([\d.]+)_', filename)
    if match:
        label = float(match.group(1))
        label=label*100
        labels.append(label)

    
    return Rh_data, labels


In [7]:
u94_Rh1_data,u94_Rh1_label = test_img('semi_anlayic/raif_img/1006_a0.42_i197.00 _rt9.13.jpg')

In [8]:
#print(u94_Rh1_data)

In [9]:
train_data=np.array(u94_Rh1_data)
label=np.array(u94_Rh1_label)

In [10]:
class RhDataset(Dataset):
    def __init__(self):
        self.x=train_data
        self.y=label

    def __getitem__(self, idx):
        
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.y)

In [11]:
test_dataset = RhDataset()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True,num_workers=0, pin_memory=True)

In [12]:
for batch in test_loader:
        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs = imgs.float()
        imgs = imgs.permute(0, 3, 1, 2)
        #imgs = torch.unsqueeze(imgs, dim=0)  # 在第0维上增加一个维度
        
        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            test_pred = model_best(imgs.to(device))

In [13]:
print('true label is ',u94_Rh1_label)
print('predict label is',test_pred.argmax(dim=-1).cpu())

true label is  [42.0]
predict label is tensor([42])
